In [21]:
import pandas as pd

# 파일 읽기

In [22]:
kapt_excel_path = "../input/20251107_단지_기본정보.xlsx"
kapt_csv_path = "../input/1_kapt.csv"

sapt_excel_path = "../input/서울시 공공주택 아파트 정보.xlsx"
sapt_csv_path = "../input/sapt.csv"

In [23]:
# kapt = pd.read_excel(kapt_excel_path)
# kapt.to_csv(kapt_csv_path)

# sapt = pd.read_excel(sapt_excel_path)
# sapt.to_csv(sapt_csv_path)

In [24]:
kapt = pd.read_csv(kapt_csv_path)
kapt = kapt[kapt['시도']=="서울특별시"]
print(f"kapt : {len(kapt)}")

sapt = pd.read_csv(sapt_csv_path)
print(f"sapt : {len(sapt)}")

kapt : 3123
sapt : 2852


/var/folders/tk/y1vzx6w13s9gmhsstd16yl_00000gn/T/ipykernel_46095/1752711429.py:1: DtypeWarning: Columns (38,41,65) have mixed types. Specify dtype option on import or set low_memory=False.
  kapt = pd.read_csv(kapt_csv_path)


# 병합

In [25]:
merged = kapt.merge(    # outer join
    sapt,
    left_on='단지코드',
    right_on='k-아파트코드',
    how='outer',
    indicator=True
)

matched = (             # 매칭에 성공한 것
    merged.loc[merged['_merge'] == 'both']
    .drop(columns = ['_merge'])
    .copy()
)

unmatched_kapt = (
    merged.loc[merged['_merge'] == 'left_only']
    .drop(columns = ['_merge'])
    .copy()
)

unmatched_sapt = (
    merged.loc[merged['_merge'] == 'right_only']
    .drop(columns = ['_merge'])
    .copy()
)

print(f"merged : {len(merged)}")
print(f"matched : {len(matched)}")
print(f"unmatched_kapt : {len(unmatched_kapt)}")
print(f"unmatched_sapt : {len(unmatched_sapt)}")

merged : 3240
matched : 2735
unmatched_kapt : 388
unmatched_sapt : 117


#### 단지코드 컬럼 병합

In [26]:
merged['단지코드'] = merged['단지코드'].fillna(merged['k-아파트코드'])

merged = merged.drop(columns=['k-아파트코드'])

#### 분양형태 컬럼 병합
kapt와 sapt의 분양형태 컬럼을 하나의 컬럼으로 병합<br>
한쪽이 NaN인 경우 다른쪽의 값 이용<br>
값이 같을 경우 하나의 컬럼으로 병합<br>
값이 다를 경우 kapt의 값 적용하고 sapt의 값은 ['보조_분양형태']컬럼 생성해서 저장.<br>
기존의 분양형태 컬럼은 삭제

In [27]:
def merge_분양형태(kapt_분양형태, sapt_분양형태):
    merged_분양형태=''  # 병합된 컬럼값
    보조_분양형태=''     # 병합 불가능한 경우, 불일치한 값 저장
    
    if(pd.isna(kapt_분양형태)): merged_분양형태 = sapt_분양형태 # NaN 예외처리
    elif(pd.isna(sapt_분양형태)): merged_분양형태 = kapt_분양형태
    elif(
        (kapt_분양형태==sapt_분양형태)
        or (kapt_분양형태=='혼합' and sapt_분양형태=='기타')
        ):
        merged_분양형태=kapt_분양형태
    else:
        merged_분양형태=kapt_분양형태
        보조_분양형태=sapt_분양형태
    return merged_분양형태, 보조_분양형태

merged = merged.rename(columns ={   # 컬럼명 직관적으로 변경
    '분양형태' : 'kapt_분양형태',
    'k-세대타입(분양형태)' : 'sapt_분양형태'
    }) 

merged[['분양형태','보조_분양형태']] = merged.apply(    # 함수 적용
    lambda row: merge_분양형태(row['kapt_분양형태'], row['sapt_분양형태']),
    axis=1,
    result_type='expand'
)

# 칼럼 위치 변경
col = merged.pop('분양형태')
merged.insert(merged.columns.get_loc('kapt_분양형태')+1,'분양형태',col)
col = merged.pop('보조_분양형태')
merged.insert(merged.columns.get_loc('kapt_분양형태')+2,'보조_분양형태',col)

merged=merged.drop(columns=['kapt_분양형태', 'sapt_분양형태']) # 병합에 쓰인 컬럼 드랍

#### 도로명주소 컬럼 병합
kapt와 sapt의 도로명주소 컬럼 병합<br>
한쪽이 NaN이거나 공백일 경우 그렇지 않은 값을 이용해 병합<br>
두 값이 같을 경우 병합<br>
kapt주소가 sapt주소를 포함하는 경우, kapt주소를 이용해 병합<br>
예시) kapt_도로명주소 = "강서로 1-1, 강서로 1-2", sapt_도로명주소 = "강서로 1-1"

In [28]:
def merge_도로명주소(kapt_도로명주소, sapt_도로명주소):
    merged_도로명주소 = ''
    보조_도로명주소 = ''
    
    if(pd.isna(kapt_도로명주소) or kapt_도로명주소==''):    # 한쪽이 NaN이거나 공백인 경우
        merged_도로명주소=sapt_도로명주소
    elif(pd.isna(sapt_도로명주소) or sapt_도로명주소==''):    
        merged_도로명주소=kapt_도로명주소
    elif(kapt_도로명주소==sapt_도로명주소):   # 값이 같은 경우
        merged_도로명주소=kapt_도로명주소
    elif(sapt_도로명주소 in kapt_도로명주소):    # kapt주소가 sapt주소를 포함하는 경우
        merged_도로명주소=kapt_도로명주소
    else:
        merged_도로명주소 = kapt_도로명주소
        보조_도로명주소 = sapt_도로명주소
    return merged_도로명주소, 보조_도로명주소

merged = merged.rename(columns={
    '도로명주소' : 'kapt_도로명주소',
    'kapt도로명주소' : 'sapt_도로명주소'
})

merged[['도로명주소', '보조_도로명주소']] = merged.apply(
    lambda row: merge_도로명주소(row['kapt_도로명주소'], row['sapt_도로명주소']),
    axis=1,
    result_type='expand'
)

col = merged.pop('도로명주소')
merged.insert(merged.columns.get_loc('kapt_도로명주소')+1,'도로명주소',col)
col = merged.pop('보조_도로명주소')
merged.insert(merged.columns.get_loc('kapt_도로명주소')+2,'보조_도로명주소',col)

merged=merged.drop(columns=['kapt_도로명주소', 'sapt_도로명주소'])

#### 단지분류 컬럼 병합
값이 같을 경우 병합<br>
다를 경우 kapt의 단지분류를 따름, sapt의 값은 ['보조_단지분류'] 컬럼을 생성해서 저장

In [29]:
def merge_단지분류 (kapt_단지분류, sapt_단지분류):
    merged_단지분류 = ''
    보조_단지분류 = ''
    
    if(pd.isna(kapt_단지분류)): merged_단지분류=sapt_단지분류
    elif(pd.isna(sapt_단지분류)): merged_단지분류=kapt_단지분류
    elif(kapt_단지분류==sapt_단지분류):
        merged_단지분류=kapt_단지분류
    else:
        merged_단지분류=kapt_단지분류
        보조_단지분류=sapt_단지분류
    return merged_단지분류, 보조_단지분류

merged = merged.rename(columns={
    '단지분류' : 'kapt_단지분류',
    'k-단지분류' : 'sapt_단지분류'
})

merged[['단지분류', '보조_단지분류']] = merged.apply(
    lambda row: merge_단지분류(row['kapt_단지분류'], row['sapt_단지분류']),
    axis=1,
    result_type='expand'
)

col = merged.pop('단지분류')
merged.insert(merged.columns.get_loc('kapt_단지분류')+1,'단지분류',col)
col = merged.pop('보조_단지분류')
merged.insert(merged.columns.get_loc('kapt_단지분류')+2,'보조_단지분류',col)

merged=merged.drop(columns=['kapt_단지분류', 'sapt_단지분류'])

#### 관리방식 컬럼 병합

In [30]:
def merge_관리방식(kapt_관리방식, sapt_관리방식):
    merged_관리방식=''
    보조_관리방식=''
    
    if(pd.isna(kapt_관리방식)): merged_관리방식=sapt_관리방식
    elif(pd.isna(sapt_관리방식)): merged_관리방식=kapt_관리방식
    elif(kapt_관리방식==sapt_관리방식):
        merged_관리방식 = kapt_관리방식
    else:
        merged_관리방식=kapt_관리방식
        보조_관리방식=sapt_관리방식
    return merged_관리방식, 보조_관리방식

merged = merged.rename(columns={
    '관리방식' : 'kapt_관리방식',
    'k-관리방식' : 'sapt_관리방식'
})

merged[['관리방식', '보조_관리방식']] = merged.apply(
    lambda row: merge_관리방식(row['kapt_관리방식'], row['sapt_관리방식']),
    axis=1,
    result_type='expand'
)

col = merged.pop('관리방식')
merged.insert(merged.columns.get_loc('kapt_관리방식')+1,'관리방식',col)
col = merged.pop('보조_관리방식')
merged.insert(merged.columns.get_loc('kapt_관리방식')+2,'보조_관리방식',col)

merged=merged.drop(columns=['kapt_관리방식', 'sapt_관리방식'])

#### 복도유형 컬럼 병합

In [31]:
def merge_복도유형(kapt_복도유형, sapt_복도유형):
    merged_복도유형=''
    보조_복도유형=''
    
    if(pd.isna(kapt_복도유형)): merged_복도유형=sapt_복도유형
    elif(pd.isna(sapt_복도유형)): merged_복도유형=kapt_복도유형
    elif(kapt_복도유형==sapt_복도유형):
        merged_복도유형 = kapt_복도유형
    else:
        merged_복도유형=kapt_복도유형
        보조_복도유형=sapt_복도유형
    return merged_복도유형, 보조_복도유형

merged = merged.rename(columns={
    '복도유형' : 'kapt_복도유형',
    'k-복도유형' : 'sapt_복도유형'
})

merged[['복도유형','보조_복도유형']] = merged.apply(
    lambda row: merge_복도유형(row['kapt_복도유형'], row['sapt_복도유형']),
    axis=1,
    result_type='expand'
)

col = merged.pop('복도유형')
merged.insert(merged.columns.get_loc('kapt_복도유형')+1,'복도유형',col)
col = merged.pop('보조_복도유형')
merged.insert(merged.columns.get_loc('kapt_복도유형')+2,'보조_복도유형',col)

merged=merged.drop(columns=['kapt_복도유형', 'sapt_복도유형'])

#### 난방방식 컬럼 병합

In [32]:
def merge_난방방식(kapt_난방방식, sapt_난방방식):
    merged_난방방식=''
    보조_난방방식=''
    
    if(pd.isna(kapt_난방방식)): merged_난방방식=sapt_난방방식
    elif(pd.isna(sapt_난방방식)): merged_난방방식=kapt_난방방식
    elif(kapt_난방방식==sapt_난방방식):
        merged_난방방식 = kapt_난방방식
    else:
        merged_난방방식=kapt_난방방식
        보조_난방방식=sapt_난방방식
    return merged_난방방식, 보조_난방방식

merged = merged.rename(columns={
    '난방방식' : 'kapt_난방방식',
    'k-난방방식' : 'sapt_난방방식'
})

merged[['난방방식','보조_난방방식']] = merged.apply(
    lambda row: merge_난방방식(row['kapt_난방방식'], row['sapt_난방방식']),
    axis=1,
    result_type='expand'
)

col = merged.pop('난방방식')
merged.insert(merged.columns.get_loc('kapt_난방방식')+1,'난방방식',col)
col = merged.pop('보조_난방방식')
merged.insert(merged.columns.get_loc('kapt_난방방식')+2,'보조_난방방식',col)

merged=merged.drop(columns=['kapt_난방방식', 'sapt_난방방식'])

#### 시공사 컬럼 병합

In [33]:
def merge_시공사(kapt_시공사, sapt_시공사):
    merged_시공사=''
    보조_시공사=''
    
    if(pd.isna(kapt_시공사) or kapt_시공사==''): 
        merged_시공사=sapt_시공사
    elif(pd.isna(sapt_시공사) or sapt_시공사==''): 
        merged_시공사=kapt_시공사
    elif(kapt_시공사==sapt_시공사):
        merged_시공사 = kapt_시공사
    else:
        merged_시공사=kapt_시공사
        보조_시공사=sapt_시공사
    return merged_시공사, 보조_시공사

merged = merged.rename(columns={
    '시공사' : 'kapt_시공사',
    'k-건설사(시공사)' : 'sapt_시공사'
})

merged[['시공사','보조_시공사']] = merged.apply(
    lambda row: merge_시공사(row['kapt_시공사'], row['sapt_시공사']),
    axis=1,
    result_type='expand'
)

col = merged.pop('시공사')
merged.insert(merged.columns.get_loc('kapt_시공사')+1,'시공사',col)
col = merged.pop('보조_시공사')
merged.insert(merged.columns.get_loc('kapt_시공사')+2,'보조_시공사',col)

merged=merged.drop(columns=['kapt_시공사', 'sapt_시공사'])

#### 시행사 컬럼 병합

In [34]:
def merge_시행사(kapt_시행사, sapt_시행사):
    merged_시행사=''
    보조_시행사=''
    
    if(pd.isna(kapt_시행사) or kapt_시행사==''): 
        merged_시행사=sapt_시행사
    elif(pd.isna(sapt_시행사) or sapt_시행사==''): 
        merged_시행사=kapt_시행사
    elif(kapt_시행사==sapt_시행사):
        merged_시행사 = kapt_시행사
    else:
        merged_시행사=kapt_시행사
        보조_시행사=sapt_시행사
    return merged_시행사, 보조_시행사

merged = merged.rename(columns={
    '시행사' : 'kapt_시행사',
    'k-시행사' : 'sapt_시행사'
})

merged[['시행사','보조_시행사']] = merged.apply(
    lambda row: merge_시행사(row['kapt_시행사'], row['sapt_시행사']),
    axis=1,
    result_type='expand'
)

col = merged.pop('시행사')
merged.insert(merged.columns.get_loc('kapt_시행사')+1,'시행사',col)
col = merged.pop('보조_시행사')
merged.insert(merged.columns.get_loc('kapt_시행사')+2,'보조_시행사',col)

merged=merged.drop(columns=['kapt_시행사', 'sapt_시행사'])

#### 경비관리-관리방식 컬럼 병합

In [35]:
def merge_경비관리방식(kapt_경비관리방식, sapt_경비관리방식):
    merged_경비관리방식=''
    보조_경비관리방식=''
    
    if(pd.isna(kapt_경비관리방식) or kapt_경비관리방식==''): 
        merged_경비관리방식=sapt_경비관리방식
    elif(pd.isna(sapt_경비관리방식) or sapt_경비관리방식==''): 
        merged_경비관리방식=kapt_경비관리방식
    elif(
        (kapt_경비관리방식=="위탁관리" and sapt_경비관리방식=="위탁")
        or (kapt_경비관리방식=="자치관리" and sapt_경비관리방식=="직영")
        ):
        merged_경비관리방식 = kapt_경비관리방식
    else:
        merged_경비관리방식=kapt_경비관리방식
        보조_경비관리방식=sapt_경비관리방식
    return merged_경비관리방식, 보조_경비관리방식

merged = merged.rename(columns={
    '경비관리-관리방식' : 'kapt_경비관리방식',
    '경비비관리형태' : 'sapt_경비관리방식'
})

merged[['경비관리방식', '보조_경비관리방식']] = merged.apply(
    lambda row: merge_경비관리방식(row['kapt_경비관리방식'], row['sapt_경비관리방식']),
    axis=1,
    result_type='expand'
)

col = merged.pop('경비관리방식')
merged.insert(merged.columns.get_loc('kapt_경비관리방식')+1,'경비관리방식',col)
col = merged.pop('보조_경비관리방식')
merged.insert(merged.columns.get_loc('kapt_경비관리방식')+2,'보조_경비관리방식',col)

merged=merged.drop(columns=['kapt_경비관리방식', 'sapt_경비관리방식'])

#### 청소관리-관리방식 컬럼 병합

In [36]:
def merge_청소관리방식(kapt_청소관리방식, sapt_청소관리방식):
    merged_청소관리방식=''
    보조_청소관리방식=''
    
    if(pd.isna(kapt_청소관리방식) or kapt_청소관리방식==''): 
        merged_청소관리방식=sapt_청소관리방식
    elif(pd.isna(sapt_청소관리방식) or sapt_청소관리방식==''): 
        merged_청소관리방식=kapt_청소관리방식
    elif(
        (kapt_청소관리방식=="위탁관리" and sapt_청소관리방식=="위탁")
        or (kapt_청소관리방식=="자치관리" and sapt_청소관리방식=="직영")
        ):
        merged_청소관리방식 = kapt_청소관리방식
    else:
        merged_청소관리방식=kapt_청소관리방식
        보조_청소관리방식=sapt_청소관리방식
    return merged_청소관리방식, 보조_청소관리방식

merged = merged.rename(columns={
    '청소관리-관리방식' : 'kapt_청소관리방식',
    '청소비관리형태' : 'sapt_청소관리방식'
})

merged[['청소관리방식','보조_청소관리방식']] = merged.apply(
    lambda row: merge_청소관리방식(row['kapt_청소관리방식'], row['sapt_청소관리방식']),
    axis=1,
    result_type='expand'
)

col = merged.pop('청소관리방식')
merged.insert(merged.columns.get_loc('kapt_청소관리방식')+1,'청소관리방식',col)
col = merged.pop('보조_청소관리방식')
merged.insert(merged.columns.get_loc('kapt_청소관리방식')+2,'보조_청소관리방식',col)

merged=merged.drop(columns=['kapt_청소관리방식', 'sapt_청소관리방식'])

In [37]:
merged.drop(columns=('_merge'))

,Unnamed: 0_x,시도,시군구,읍면,동리,단지코드,단지명,단지분류,보조_단지분류,법정동주소,...,Unnamed: 49,Unnamed: 50,Unnamed: 51,Unnamed: 52,Unnamed: 53,Unnamed: 54,Unnamed: 55,Unnamed: 56,Unnamed: 57,Unnamed: 58
0,NaN,NaN,NaN,NaN,NaN,A10020326,NaN,아파트,,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,A10020345,NaN,주상복합,,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,A10020346,NaN,아파트,,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,A10020358,NaN,아파트,,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,9179.0,서울특별시,광진구,NaN,중곡동,A10020402,중곡아파트2단지,아파트,,서울특별시 광진구 중곡동 190-26 중곡아파트2단지,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3235,NaN,NaN,NaN,NaN,NaN,B20160929,NaN,NaN,,NaN,...,2019-08-28 19:43:21,Y,N,127.0952057887,NaN,NaN,NaN,NaN,NaN,NaN
3236,NaN,NaN,NaN,NaN,NaN,B20161019,NaN,NaN,,NaN,...,Y,N,127.097182157,37.61,NaN,NaN,NaN,NaN,NaN,NaN
3237,NaN,NaN,NaN,NaN,NaN,B20161027,NaN,NaN,,NaN,...,127.133588053,37.536758671,2016-10-24 10:34:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3238,NaN,NaN,NaN,NaN,NaN,B20161103,NaN,NaN,,NaN,...,126.822877,37.559259,2016-11-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# 디자인 추가

In [38]:
def highlight_merge_fail(row, main_col, sub_col, color="#ffd900"):
    styles = [''] * len(row)

    if pd.notna(row[sub_col]) and row[sub_col] != '':
        for i, col in enumerate(row.index):
            if col in [main_col, sub_col]:
                styles[i] = f'background-color: {color}'

    return styles

In [39]:
styled = merged.style

styled = styled.apply(
    highlight_merge_fail,
    axis=1,
    main_col='분양형태',
    sub_col='보조_분양형태'
)

styled = styled.apply(
    highlight_merge_fail,
    axis=1,
    main_col='도로명주소',
    sub_col='보조_도로명주소'
)

styled = styled.apply(
    highlight_merge_fail,
    axis=1,
    main_col='단지분류',
    sub_col='보조_단지분류'
)

styled = styled.apply(
    highlight_merge_fail,
    axis=1,
    main_col='관리방식',
    sub_col='보조_관리방식'
)

styled = styled.apply(
    highlight_merge_fail,
    axis=1,
    main_col='복도유형',
    sub_col='보조_복도유형'
)

styled = styled.apply(
    highlight_merge_fail,
    axis=1,
    main_col='난방방식',
    sub_col='보조_난방방식'
)

styled = styled.apply(
    highlight_merge_fail,
    axis=1,
    main_col='시공사',
    sub_col='보조_시공사'
)

styled = styled.apply(
    highlight_merge_fail,
    axis=1,
    main_col='시행사',
    sub_col='보조_시행사'
)

styled = styled.apply(
    highlight_merge_fail,
    axis=1,
    main_col='경비관리방식',
    sub_col='보조_경비관리방식'
)

styled = styled.apply(
    highlight_merge_fail,
    axis=1,
    main_col='청소관리방식',
    sub_col='보조_청소관리방식'
)

# 파일 저장

In [40]:
styled.to_excel("../output/kapt_sapt_병합됨.xlsx", index = False)
unmatched_kapt.to_excel("../output/kapt_병합실패.xlsx", index = False)
unmatched_sapt.to_excel("../output/sapt_병합실패.xlsx", index = False)